<a href="https://colab.research.google.com/github/pranay-aggarwal/Minor-Project./blob/main/Minor_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bing_image_downloader

In [ ]:
!pip install streamlit
!pip install pyngrok


In [ ]:
!mkdir images

In [ ]:
from bing_image_downloader import downloader
downloader.download('cats',limit=30,output_dir='images',adult_filter_off=True)

In [ ]:
downloader.download('dogs',limit=30,output_dir='images',adult_filter_off=True)

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import accuracy_score,confusion_matrix
import pickle
from pyngrok import ngrok
import streamlit as st

In [ ]:
target=[]
images=[]
flat_data=[]

DATADIR= '/content/images'
CATEGORIES=['cats','dogs']



In [ ]:
for category in CATEGORIES:
    class_num=CATEGORIES.index(category)
    path=os.path.join(DATADIR,category)
    for img in os.listdir(path):
       img_array=imread(os.path.join(path,img))
       img_resized=resize(img_array,(150,150,3))
       flat_data.append(img_resized.flatten())
       images.append(img_resized)
       target.append(class_num)

flat_data=np.array(flat_data)
target=np.array(target)
images=np.array(images)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(flat_data,target,test_size=0.3,random_state=109)

In [ ]:
param_grid= [
             {'C':[1.0,10.0,100.0,1000.0],'gamma':[0.001,0.0001],'kernel':['rbf']},
              {'C':[1.0,10.0,100.0,1000.0],'kernel':['linear']},
]

svc=svm.SVC(probability=True)
clf=GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

In [ ]:
y_pred=clf.predict(x_test)
y_pred

In [ ]:
y_test

In [ ]:
accuracy_score(y_pred,y_test)

In [ ]:
confusion_matrix(y_pred,y_test)

In [ ]:
pickle.dump(clf,open('img_model.p','wb'))
model=pickle.load(open('img_model.p','rb'))
flat_data=[]
url=input('enter your url of image: ')
img= imread(url)
img_resize=resize(img,(150,150,3))
flat_data.append(img_resize.flatten())
flat_data=np.array(flat_data)
print(img.shape)
y_out=model.predict(flat_data)
y_out=CATEGORIES[y_out[0]]
print(f' predicted output: {y_out} ')  

In [ ]:
%%writefile app.py  
import streamlit as st
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import pickle
from PIL import Image
st.title('Image Classifier- Cats or Dogs')
st.text('upload the image')
model=pickle.load(open('img_model.p','rb'))
uploaded_file =st.file_uploader('choose an image...',type='jpg')
if uploaded_file is not None:
    img=Image.open(uploaded_file)
    st.image(img,caption'uploaded file')
if st.button('PREDICT'):
    CATEGORIES=['cats','dogs']
    st.write('result.....')
    flat_data=[]
    img=np.array(img)
    img_resize=resize(img,(150,150,3))
    flat_data.append(img_resize.flatten())
    flat_data=np.array(flat_data)
    y_out=model.predict(flat_data)
    y_out=CATEGORIES[y_out[0]]
    st.title(f' predicted output: {y_out} ')
    q=model.predict_proba(flat_data)
    for index,item in enumerate(CATEGORIES):
        st.write(f'{item}:{q[0][index]*100%}')

In [ ]:
!nohup streamlit run app.py &
 
url=ngrok.connect(port='8501')
url

In [ ]:
##sample images
#https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSfx__RoRYzLDgXDiJxYGxLihJC4zoqV3V0xg&usqp=CAU.jpg
#https://specials-images.forbesimg.com/imageserve/6082931ef598a85b055afe77/960x0.jpg?cropX1=0&cropX2=3475&cropY1=182&cropY2=2137.jpg